# Notebook for data cleaning

In [ ]:
import pandas as pd
from unidecode import unidecode

In [ ]:
df_main_raw = pd.read_parquet("data/properties_main.parquet", engine="pyarrow", dtype_backend="numpy_nullable")
df_features = pd.read_parquet("data/properties_characteristics.parquet")


In [ ]:
df_main = df_main_raw.copy()
df_main["city"] = df_main_raw["locality"].apply(lambda row: row.split(",")[1])
df_main["neighborhood"] = df_main_raw["locality"].apply(lambda row: unidecode(row.split(",")[0]).upper().strip())
df_main = df_main.drop(columns=["locality"])
sao_carlos = df_main[df_main["city"] == "São Carlos"]

In [ ]:
# Droping columns not to be put in the model (id is kept to re-join if needed)
# Also selecting only properties from sao carlos

selected = (sao_carlos
.drop(columns=[
    "property_reference",
    "title",
    'description',
    'postal_code',
    'address',
    'latitude',
    'longitude', # A more complex model may use it to calculate distance to uptown, will not be used at first
    'city',
    'neighborhood', # Using neighborhoods may introduce too much sparsity in the model (over 200 neighborhoods)
    'show_map',
    'has_sale_price',
    'has_rent_price', # Already visible by NaNs
    'image_count',
    'publisher_code',
    'publisher_name',
    'publisher_phone',
    'price_per_sqm_rent',
    'price_per_sqm_sale'
]))

In [ ]:
# We will separate the residencial properties to use in the model
residencial_df = selected[selected['property_type'].isin(["Casa", "Apartamento"])]

In [ ]:
# lets look at nan values
pd.DataFrame(residencial_df.isnull().sum().divide(len(residencial_df)).sort_values(ascending=False)).T
pass

Most columns have small proportions of missing values, which can be discarded safely (also seem to be misinputs or comercial rooms listed as houses). Among the ones with high proportions:
- sale_price/rent_price -> Self explanatory, will be divided in two datasets
- condominium_fee -> non condominium houses, can be set to 0
- property_tax -> needs exploration, may be removed
- total_montlhy_cost -> way to many NaNs, will be removed
- suites -> Probably equivalent to 0 or just not included (because half of all properties dont have it), may be removed

In [ ]:
exploration_df = sao_carlos[sao_carlos['property_type'].isin(["Casa", "Apartamento"])]
exploration_df[exploration_df['condominium_fee'].isnull()]
pass

Decisions:
- Excluded columns: total_monthly_cost, suites, property_tax. Complex mixture of misinput and systematic missing, best to avoid using.
- Drop NA columns: bathrooms, bedroom, area_util, area_total, size_category, parking_spaces. Misinput or invalid data (comercial rooms), sub 5% proportion of NaNs (8% for parking_spaces) allows us to drop these rows.
- Dataset split: rent_price, sale_price.
- Fill: condominium_fee. Most are houses or kitnets, will just fill with 0, will cause some wrong data for sure, but the rate of misinput is surely way smaller than the no fee places.

In [ ]:
# Based on the EDA notebook, we shall remove nonsense outliers from the area_util column
residencial_df = residencial_df[residencial_df['area_util'] > 10]

In [ ]:
clean_data = residencial_df.copy()
clean_data = clean_data.dropna(subset=['bathrooms', 'bedrooms', 'area_total', 'area_util', 'size_category', 'parking_spaces'])
clean_data["condominium_fee"] = clean_data["condominium_fee"].fillna(0)
clean_data = clean_data.drop(columns = ['total_monthly_cost', 'suites', 'property_tax'])
clean_data_sell = clean_data[~clean_data["sale_price"].isnull()].drop(columns=["rent_price"])
clean_data_rent = clean_data[~clean_data["rent_price"].isnull()].drop(columns=["sale_price"])
len(clean_data_rent), len(clean_data_sell)
pass

In [ ]:
# save to csv
clean_data_rent.to_csv("data/clean_data_rent.csv", index=False)
clean_data_sell.to_csv("data/clean_data_sell.csv", index=False)